In [63]:
# import Dependencies
import pandas as pd

In [64]:
# Create path with a variable 
heroes_path = "../Resources/purchase_data.csv"

In [65]:
# Read in the csv from the path
heroes_df = pd.read_csv(heroes_path)
heroes_df.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [66]:
# Check to see if there is any missing data that needs to be removed
heroes_df.count()

Purchase ID    780
SN             780
Age            780
Gender         780
Item ID        780
Item Name      780
Price          780
dtype: int64

In [67]:
# look up sn's unique value
heroes_df["SN"].describe()

count           780
unique          576
top       Lisosia93
freq              5
Name: SN, dtype: object

In [147]:
# get sn's len from unique value
total_players = len(heroes_df["SN"].unique())
total_players

576

In [148]:
# Create dataframe for total players with sn_count variable as to value
total_players_df = pd.DataFrame({"Total Players": [total_players]})
total_players_df

,Total Players
0,576


In [70]:
# Get unique items count with len() and unique() fucntion
unique_items_count = len(heroes_df["Item ID"].unique())
unique_items_count

179

In [71]:
# Get average from Price column using the mean() function
average = heroes_df["Price"].mean()
average

3.0509871794871795

In [72]:
# Get total number of purchases with len() function 
total_purchases = len(heroes_df)
total_purchases

780

In [73]:
# Get total revenue from Price column using sum() function
total_revenue = heroes_df["Price"].sum()
total_revenue

2379.77

In [74]:
# Create dataframe using a dictionary with keys (header) and values collected above
purchasing_analysis_df = pd.DataFrame({"Number of Unique Items": [unique_items_count], "Average Price": [average], 
                                       "Number of Purchases": [total_purchases], "Total Revenue": [total_revenue] })
purchasing_analysis_df

,Number of Unique Items,Average Price,Number of Purchases,Total Revenue
0,179,3.050987,780,2379.77


In [171]:
group_by_gender = heroes_df.groupby("Gender")
gender_count = group_by_gender["SN"].nunique()
percentage = (gender_count / total_players) * 100

gender_table_df = pd.DataFrame({"Total Count": gender_count, "Percentage of Players": percentage})
gender_table_df

,Total Count,Percentage of Players
Gender,,
Female,81,14.062500
Male,484,84.027778
Other / Non-Disclosed,11,1.909722


In [169]:
group_by_gender = heroes_df.groupby("Gender")
gender_count = group_by_gender["SN"].nunique()
purchase_count = group_by_gender["Price"].count()
total_value_per_gender = group_by_gender["Price"].sum()
average_purchase_price = (total_value_per_gender / purchase_count)
average_per_person = (total_value_per_gender / gender_count)

In [172]:
gender_demographics_df = pd.DataFrame({"Purchase Count": purchase_count, "Average Purchase Price": average_purchase_price, 
                                      "Total Purchase Value": total_value_per_gender, 
                                       "Avg Total Purchase per Person": average_per_person})
gender_demographics_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Avg Total Purchase per Person
Gender,,,,
Female,113,3.203009,361.94,4.468395
Male,652,3.017853,1967.64,4.065372
Other / Non-Disclosed,15,3.346000,50.19,4.562727


In [183]:
# Establish bins for ages
age_bin = [0, 10, 15, 20, 25, 30 ,35, 40]
bin_name = ["<10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-39", "40+"]


In [184]:
age = heroes_df.groupby("Age")
age_count = age["SN"].nunique()
age_purchase_count = age["Price"].count()
age_total_value = age["Price"].sum()
age_avg_purchase_price = (age_total_value / age_purchase_count)
age_avg_per_person = (age_total_value / age_count)

In [186]:
age_demographic_df = pd.DataFrame({"Total Count": age_count, "Percentage of Players": })

TypeError: 'DataFrameGroupBy' object is not callable

In [81]:
#create a groupby with SN and gender since we need to get group_sn_df = heroes_df.groupby("SN")["Gender"].count()
#confirmed that this dataframes length is the same as the total player amount
gender_table = pd.DataFrame(group_sn_df)
gender_table

,0,1
0,Female,15 Lisassa64 18 Reunasu60 ...
1,Male,0 Lisim78 1 Lisovynya38 2 ...
2,Other / Non-Disclosed,9 Chanosian48 22 Siarithria38 82 ...
